Step 23: Compute and match crop embeddings

In [ ]:
for det in detections:
    img = preprocess(Image.open(det["crop_path"]).convert("RGB")).unsqueeze(0).to(device)

    with torch.no_grad():
        emb = clip_model.encode_image(img)
        emb = emb / emb.norm(dim=-1, keepdim=True)

    emb = emb.cpu().numpy().astype("float32")
    D, I = index.search(emb, 1)

    similarity = D[0][0]
    matched_idx = I[0][0]
    matched_product_id = catalog_product_ids[matched_idx]

    if similarity > 0.9:
        match_type = "Exact Match"
    elif similarity > 0.75:
        match_type = "Similar Match"
    else:
        match_type = "No Match"

    det.update({
        "matched_product_id": matched_product_id,
        "match_type": match_type,
        "similarity": float(similarity)
    })